In [ ]:
import duckdb
from dotenv import find_dotenv, load_dotenv
import os


load_dotenv(find_dotenv())

In [ ]:
if os.getenv("SNOWFLAKE_PASSWORD") is None:
    raise ValueError("SNOWFLAKE_PASSWORD is not set")

if os.getenv("SA_USER") is None:
    raise ValueError("SA_USER is not set")

if os.getenv("SNOWFLAKE_ACCOUNT_URL") is None:
    raise ValueError("SNOWFLAKE_ACCOUNT_URL is not set")

pat_token = os.getenv("SNOWFLAKE_PASSWORD")
snowflake_account_url = os.getenv("SNOWFLAKE_ACCOUNT_URL")
catalog_uri = f"{os.getenv("SNOWFLAKE_ACCOUNT_URL")}/polaris/api/catalog".lower()
role = os.getenv("SA_ROLE")
warehouse = os.getenv("SNOWFLAKE_DATABASE","KAMESH_DEMOS")

In [ ]:
secret_sql = f"""
  CREATE SECRET iceberg_pat_secret (
  TYPE iceberg,
  CLIENT_ID '',
  CLIENT_SECRET '{pat_token}',
  OAUTH2_SERVER_URI '{catalog_uri}/v1/oauth/tokens',
  OAUTH2_GRANT_TYPE 'client_credentials',
  OAUTH2_SCOPE 'session:role:{role}'
  );
"""

print(secret_sql)

attach_params = [
  "TYPE iceberg",
  "SECRET iceberg_pat_secret",
  f"ENDPOINT '{catalog_uri}'"
]

attach_sql = f"""
  ATTACH '{warehouse}' AS {warehouse} (
    {', '.join(attach_params)}
  );
"""

In [ ]:
import traceback

try:
    conn = duckdb.connect()
    conn.execute("SET enable_http_logging = true;")
    conn.execute("INSTALL iceberg;")
    conn.execute("LOAD iceberg;")
    conn.execute("INSTALL httpfs;")
    conn.execute("LOAD httpfs;")

    conn.execute(secret_sql)
    conn.execute(attach_sql)

    tables_result = conn.execute("SHOW ALL TABLES;").fetchall()

    first_table = tables_result[0]
except Exception as e:
    traceback.print_exc()

# database, schema, name, column_names, column_types, temporary = first_table

# full_table_name = f"{database}.{schema}.{name}"

# sample_query = f"SELECT * FROM {full_table_name} LIMIT 5;"

# sample_result = conn.execute(sample_query).fetchall()

# for i, row in enumerate(sample_result[:3]): # Show first 3 rows
#   print(f" Row {i+1}: {row}")